# Open-ended Capstone Step 4: Data Exploration

The main goal of the project is to be able to evaluate the success of each product in the different marketplaces (units sold per product per sales channel). At this phase of the project, we explore part of our dataset to start identifying what cleaning and transformation steps will be required for production.

One of the key tables of our MySQL database and the core element of the Dimensional Star Schema development is the Sales Order table, which represents the sales transactions (at order level, not item level). 

**Dataset Exploration Steps:**

1. Pandas library is imported to read and manipulate CSV files as DataFrames. CSV files were created from MySQL database (transactional system), using Jupyter Notebook dataset_extraction.ipynb. 

In [300]:
import pandas as pd

2. Table Sales Orders (2021 transactions) is read into a DataFrame using Pandas.

    The total number of sales transactions is displayed.

In [301]:
so = pd.read_csv("C:\\Users\\FBLServer\\Documents\\Python Scripts\\SB\\so_reduced.csv")
print("Number of sales transactions in 2021 is: " + str(len(so.index)))

Number of sales transactions in 2021 is: 2320


3. Dataset datatypes are displayed. 

We can see that date fields are read as object data type (mixed numeric and non-numeric values). Since we will be using Spark for transformation in production and MySQL tables will be read directly from database, we will have access to the schema. For practicality, we keep the datatypes as they are for now.

In [302]:
print("Datatypes of each column are:")
print()
print(so.dtypes)

Datatypes of each column are:

id                   int64
currencyId         float64
customerId           int64
dateCompleted       object
dateCreated         object
locationGroupId      int64
qbClassId            int64
statusId             int64
dtype: object


4. Exploring a few rows (most recent transactions). 

    Fields are:

Field|Description
:----|:----------
id|Unique ID of Sales Order
currencyId|Currency used in transaction
customerId|Unique ID of customer
dateCompleted|Date when order was fulfilled
dateCreated|Date when order was created
locationGroupId|Warehouse ID
qbClassId|Accounting class ID
statusId| Sales Order status

In [303]:
so.tail(50)

,id,currencyId,customerId,dateCompleted,dateCreated,locationGroupId,qbClassId,statusId
2270,16187,NaN,6320,2021-06-09 16:08:30.410,2021-06-09 00:00:00.000,1,9,60
2271,16188,NaN,2466,NaN,2021-05-07 00:00:00.000,1,12,10
2272,16189,NaN,559,NaN,2021-05-21 00:00:00.000,1,12,10
2273,16190,NaN,6480,2021-06-09 16:01:53.451,2021-06-09 00:00:00.000,1,9,60
2274,16191,NaN,559,NaN,2021-05-21 00:00:00.000,1,12,10
2275,16192,NaN,312,2021-06-09 15:57:25.667,2021-06-08 00:00:00.000,1,9,60
2276,16193,NaN,312,2021-06-09 15:57:56.232,2021-06-09 00:00:00.000,1,9,60
2277,16194,NaN,3019,2021-06-10 16:25:36.266,2021-06-09 00:00:00.000,1,9,60
2278,16195,NaN,1,2021-06-10 16:23:48.739,2021-06-09 00:00:00.000,1,8,60
2279,16196,NaN,1731,2021-06-10 16:20:29.685,2021-06-09 00:00:00.000,1,10,60


5. As we can see, columns "currencyId" and "dateCompleted" show NaN values. 

    In the case of "currencyId", only when the order is in Euro currency, it shows a value of 6.0. For any order in USD currency, it shows a value of NaN. NaN value is corrected by replacing it with another ID (using 1.0 to represent USD).

In [304]:
so["currencyId"] = so["currencyId"].fillna(1.0)
so.tail(10)

,id,currencyId,customerId,dateCompleted,dateCreated,locationGroupId,qbClassId,statusId
2310,16228,1.0,734,NaN,2021-06-12 00:00:00.000,1,9,20
2311,16229,1.0,734,NaN,2021-06-12 00:00:00.000,1,9,20
2312,16230,1.0,734,NaN,2021-06-12 00:00:00.000,1,9,20
2313,16231,1.0,6490,NaN,2021-06-12 00:00:00.000,1,9,20
2314,16232,1.0,734,NaN,2021-06-12 00:00:00.000,1,9,20
2315,16233,1.0,6491,NaN,2021-06-12 00:00:00.000,1,9,20
2316,16234,1.0,6475,NaN,2021-06-12 00:00:00.000,1,9,20
2317,16235,1.0,6120,NaN,2021-06-13 00:00:00.000,1,9,20
2318,16236,1.0,6492,NaN,2021-06-13 00:00:00.000,1,9,20
2319,16237,1.0,5967,NaN,2021-06-13 00:00:00.000,1,9,20


6. As for "dateCompleted" field, every order that has not being fulfilled yet, shows a NaN value as well. Since the current design of the Dimensional Star Schema will only include orders that have been fulfilled, these rows will possibly not be extracted. For now, these rows are filtered out. Number of sales transactions fulfilled in 2021 is displayed.

In [305]:
fulfilled_so = so[so.dateCompleted.notnull()]
print("Number of sales transactions fulfilled in 2021 is: " + str(len(fulfilled_so.index)))
fulfilled_so.tail(10)

Number of sales transactions fulfilled in 2021 is: 2172


,id,currencyId,customerId,dateCompleted,dateCreated,locationGroupId,qbClassId,statusId
2283,16200,1.0,1,2021-06-10 16:25:29.704,2021-06-10 00:00:00.000,1,8,60
2285,16202,1.0,2839,2021-06-10 16:26:54.705,2021-06-09 00:00:00.000,1,10,60
2286,16203,1.0,2839,2021-06-10 16:27:20.751,2021-06-10 00:00:00.000,1,10,60
2287,16204,1.0,1731,2021-06-10 16:20:44.841,2021-06-10 00:00:00.000,1,10,60
2290,16208,1.0,6342,2021-06-11 16:41:01.904,2021-06-10 00:00:00.000,1,9,60
2295,16213,1.0,4808,2021-06-11 16:41:36.797,2021-06-10 00:00:00.000,1,9,60
2303,16221,1.0,2839,2021-06-11 16:40:12.302,2021-06-11 00:00:00.000,1,10,60
2304,16222,1.0,2839,2021-06-11 16:40:18.174,2021-06-11 00:00:00.000,1,10,60
2305,16223,1.0,3995,2021-06-11 16:38:55.398,2021-06-11 00:00:00.000,1,10,60
2306,16224,1.0,6487,2021-06-11 16:41:13.385,2021-06-08 00:00:00.000,1,9,60


7. Column "statusId" shows the status of the sales order. In this step we validate that only orders with status Fulfilled or Closed Short are left in the table. Dataset should not include orders at any other status. The different status are:

Status|Description
:----|:----------
85|Cancelled
70|Closed Short
10|Estimate
90|Expired
60|Fulfilled
95|Historical
25|In Progress
20|Issued
80|Voided


In [306]:
print(fulfilled_so["statusId"].unique())

[60 70]


8. In order to create categories that will represent the different Sales Channels, an empty column called "sales_channels" is added to the DataFrame.

In [307]:
categorized_so = fulfilled_so.assign(sales_channel="")
categorized_so.tail(10)

,id,currencyId,customerId,dateCompleted,dateCreated,locationGroupId,qbClassId,statusId,sales_channel
2283,16200,1.0,1,2021-06-10 16:25:29.704,2021-06-10 00:00:00.000,1,8,60,
2285,16202,1.0,2839,2021-06-10 16:26:54.705,2021-06-09 00:00:00.000,1,10,60,
2286,16203,1.0,2839,2021-06-10 16:27:20.751,2021-06-10 00:00:00.000,1,10,60,
2287,16204,1.0,1731,2021-06-10 16:20:44.841,2021-06-10 00:00:00.000,1,10,60,
2290,16208,1.0,6342,2021-06-11 16:41:01.904,2021-06-10 00:00:00.000,1,9,60,
2295,16213,1.0,4808,2021-06-11 16:41:36.797,2021-06-10 00:00:00.000,1,9,60,
2303,16221,1.0,2839,2021-06-11 16:40:12.302,2021-06-11 00:00:00.000,1,10,60,
2304,16222,1.0,2839,2021-06-11 16:40:18.174,2021-06-11 00:00:00.000,1,10,60,
2305,16223,1.0,3995,2021-06-11 16:38:55.398,2021-06-11 00:00:00.000,1,10,60,
2306,16224,1.0,6487,2021-06-11 16:41:13.385,2021-06-08 00:00:00.000,1,9,60,


9. Based on different conditions, the sales channel for each transaction is added to the "sales_channel" column

In [308]:
categorized_so.at[categorized_so.customerId == 3738, "sales_channel"] = "C"
categorized_so.at[categorized_so.customerId == 1731, "sales_channel"] = "G"
categorized_so.at[categorized_so.customerId == 1, "sales_channel"] = "H"
categorized_so.at[categorized_so.customerId == 6342, "sales_channel"] = "J"
categorized_so.at[categorized_so.customerId == 3806, "sales_channel"] = "N"
categorized_so.at[categorized_so.customerId == 4854, "sales_channel"] = "A"
categorized_so.at[categorized_so.customerId == 3995, "sales_channel"] = "B"
categorized_so.at[categorized_so.customerId == 426, "sales_channel"] = "E"
categorized_so.at[categorized_so.customerId == 312, "sales_channel"] = "F"
categorized_so.at[categorized_so.customerId == 2839, "sales_channel"] = "I"
categorized_so.at[categorized_so.customerId == 3809, "sales_channel"] = "M"
categorized_so.at[categorized_so.customerId == 6343, "sales_channel"] = "Q"
categorized_so.at[categorized_so.customerId == 3188, "sales_channel"] = "R"
categorized_so.at[(categorized_so['qbClassId'] == 19) & (categorized_so['customerId'] != 3738) & (categorized_so['customerId'] != 3806),"sales_channel"] = "K"
categorized_so.at[(categorized_so['qbClassId'] == 9) & (categorized_so['customerId'] != 312) & (categorized_so['customerId'] != 3809) & (categorized_so['customerId'] != 6342), "sales_channel"] = "O"
categorized_so.at[(categorized_so['qbClassId'] == 12) | (categorized_so['qbClassId'] == 17),"sales_channel"] = "P"

categorized_so.tail(10)

,id,currencyId,customerId,dateCompleted,dateCreated,locationGroupId,qbClassId,statusId,sales_channel
2283,16200,1.0,1,2021-06-10 16:25:29.704,2021-06-10 00:00:00.000,1,8,60,H
2285,16202,1.0,2839,2021-06-10 16:26:54.705,2021-06-09 00:00:00.000,1,10,60,I
2286,16203,1.0,2839,2021-06-10 16:27:20.751,2021-06-10 00:00:00.000,1,10,60,I
2287,16204,1.0,1731,2021-06-10 16:20:44.841,2021-06-10 00:00:00.000,1,10,60,G
2290,16208,1.0,6342,2021-06-11 16:41:01.904,2021-06-10 00:00:00.000,1,9,60,J
2295,16213,1.0,4808,2021-06-11 16:41:36.797,2021-06-10 00:00:00.000,1,9,60,O
2303,16221,1.0,2839,2021-06-11 16:40:12.302,2021-06-11 00:00:00.000,1,10,60,I
2304,16222,1.0,2839,2021-06-11 16:40:18.174,2021-06-11 00:00:00.000,1,10,60,I
2305,16223,1.0,3995,2021-06-11 16:38:55.398,2021-06-11 00:00:00.000,1,10,60,B
2306,16224,1.0,6487,2021-06-11 16:41:13.385,2021-06-08 00:00:00.000,1,9,60,O


10. Every transaction that was not categorized is either a closed sales channel or a sample transaction. These rows are removed from the dataset as they are considered noise. Number of meaningful sales transactions is displayed:

In [309]:
categorized_so = categorized_so[(categorized_so["sales_channel"]) != ""]
print("Number of cleaned sales transactions in 2021 is: " + str(len(categorized_so.index)))

Number of cleaned sales transactions in 2021 is: 2143


11. To get an idea of what channels handle more volume of orders, these are grouped by channel and sorted from highest to lowest.

In [310]:
sales_by_channel = categorized_so.groupby("sales_channel")['id'].count().sort_values(ascending=False)
print(sales_by_channel)
print(type(sales_by_channel))

sales_channel
H    690
O    466
K    295
G    207
I    157
F    147
P     47
C     44
N     30
B     22
E     15
J     11
R      8
A      2
M      2
Name: id, dtype: int64
<class 'pandas.core.series.Series'>


12. A summary table is created to show what is the percentage of sales orders per channel. We can see that 5 channels (H, O, K, G and I) represent almost 85% of the total sales in 2021 (in order volume).

In [311]:
sales_summary = pd.DataFrame({"sales_channels":sales_by_channel.index, "orders":sales_by_channel.values})
total_orders = sales_summary['orders'].sum()
sales_summary["%_out_of_total"] = round(sales_summary["orders"] / total_orders * 100, 2)
print(sales_summary)

   sales_channels  orders  %_out_of_total
0               H     690           32.20
1               O     466           21.75
2               K     295           13.77
3               G     207            9.66
4               I     157            7.33
5               F     147            6.86
6               P      47            2.19
7               C      44            2.05
8               N      30            1.40
9               B      22            1.03
10              E      15            0.70
11              J      11            0.51
12              R       8            0.37
13              A       2            0.09
14              M       2            0.09


Even when we can see what channels handle more volume of orders at this moment, this level of analysis does not show what products have a better performance (units sold per sales channel). The table use for this exploration will be complemented with other tables using Pyspark, in order to get to the item level analysis.

13. The categorized sales transactions are saved to disk.



In [312]:
categorized_so.to_csv("C:\\Users\\FBLServer\\Documents\\Python Scripts\\SB\\categorized_so.csv", index = False)